<a href="https://colab.research.google.com/github/ticklecatisback/My-colab-Collections/blob/main/CVPR2022_DaGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deepfakes with the CVPR2022-DaGAN
This notebook is a demo for Creating uncanny deepfakes 
* Github: https://github.com/harlanhong/CVPR2022-DaGAN.git
* Paper: https://arxiv.org/abs/2203.06605

In [ ]:
#@title STEP1: Setup
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/sudo-ken/FFmpeg-for-GDrive/master/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")
 
from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')

!git clone https://github.com/harlanhong/CVPR2022-DaGAN.git
%cd CVPR2022-DaGAN
!pip install -r requirements.txt
!pip install numba

## Install the Face Alignment lib
%cd face-alignment
!pip install -r requirements.txt
!python setup.py install
!pip install imageio-ffmpeg
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip3 install torch torchvision torchaudio
!pip install opencv-python
!pip install scipy>=0.17.0
!pip install scikit-image
!pip install numba

from IPython.display import HTML
from base64 import b64encode
def showVideo(path):
  mp4 = open(str(path),'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=700 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

from IPython.display import clear_output

In [ ]:
#@title STEP2: Download Dagen model
from zipfile import ZipFile
print("Follow this link:")
print("https://drive.google.com/open?id=1QHqh6CJB5e9lG0LPz8EIHtEKDDBH4Q3T")
print("and click 'Add shortcut to Drive' (top right of the screen)\n")
print("After that, follow the link below, select the account where you saved the pretrained model,")
print("click the 'Allow' button.")
from google.colab import drive
drive.mount('/content/gdrive')

with ZipFile('/content/gdrive/MyDrive/CVPR22_DaGAN.zip', 'r') as zipOBj:
  zipOBj.extractall('/content/CVPR2022-DaGAN/depth')

In [ ]:
#@title STEP3: Select a Image
image_url = 'https://media.bizj.us/view/img/11965652/glennfukuda-4196c-pr*1500xx1819-2425-61-0.jpg' #@param {type:"string"}
import shutil
import cv2
from IPython.display import Image

if image_url:
  !wget "$image_url" -O /content/animate.png

Image('/content/animate.png')

In [ ]:
#@title STEP3+: Upload your Image
import os
from google.colab import files
import cv2
import shutil
from IPython.display import Image
uploaded = files.upload()
dst ='putin.png' 
os.rename(list(uploaded.keys())[0], dst)

Image('/content/putin.png')

In [4]:
#@title Delete old input Image file
%rm /content/animate.png
from IPython.display import clear_output 
clear_output()

In [ ]:
#@title STEP4: Select a Youtube Video
video_url = 'https://www.youtube.com/watch?v=vAnWYLTdvfY' #@param {type:"string"}

import shutil

import os
import os.path
from IPython.display import clear_output
import youtube_dl

loadingAn(name="lds")
textAn("Downloading Video...", ty='twg')
if video_url:
  !rm -f /content//fake.mp4
  !youtube-dl -f "bestvideo[ext=mp4][vcodec!*=av01][height<=360]+bestaudio[ext=m4a]/mp4[height<=360][vcodec!*=av01]/mp4[vcodec!*=av01]/mp4" "$video_url" --merge-output-format mp4 -o /content/fake.mp4

# cut the video
textAn("Trimming Video...", ty='twg')

start_seconds = 0 #@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)

if duration_seconds:
  !mv /content/fakee.mp4 /content/full_video.mp4
  !ffmpeg -ss $start_seconds -t $duration_seconds -i /content/full_video.mp4 -f mp4 /content/wav2lip-hq/fake.mp4 -y

!rm -df youtube.mp4
# download the youtube with the given ID

#delete video.mp4 if already exits
!rm -f /content/CVPR2022-DaGAN/fake.mp4


#Preview trimmed video
clear_output()
print("Trimmed Video")
showVideo('/content/fake.mp4')

In [ ]:
#@title STEP4+: Upload your Video File
%cd /content
from google.colab import files
uploaded = files.upload()
dst ='fake.mp4' 
os.rename(list(uploaded.keys())[0], dst)

loadingAn(name="lds")
textAn("Moving Video...", ty='twg')

import os, sys, re
video_file_path = "/content/fake.mp4"

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mp4

PATH_TO_YOU_VIDEO = '/content/fake.mp4'
import subprocess

#@markdown ### Trim the video (start, end) seconds
#@markdown <i>Don't want to trim ? put <b>'start'</b> = -1 and <b>'end'</b> = -1</i>
start =  -1#@param {type:"integer"}
end =  -1#@param {type:"integer"}


#@markdown <i>Note: the trimmed video must have face on all frames</i>

# delete start end
interval = end - start

textAn("Triming Video...", ty='twg')

#delete if file already exists
!rm -f '/content/CVPR2022-DaGAN/fake.mp4'

if start < 0 or end < 0:
  #convert the video to specif location
  !ffmpeg -i '$PATH_TO_YOU_VIDEO' '/content/fake.mp4'
else:
  # cut the video
  !ffmpeg -i '$PATH_TO_YOU_VIDEO' -ss {start} -t {interval} -async 1 '/content/fake.mp4'
from IPython.display import clear_output

# Preview trimmed video
clear_output()
print("Input Video")
showVideo('/content/fake.mp4')

In [ ]:
#@title Delete old input Video file
%rm /content/fake.mp4
from IPython.display import clear_output 
clear_output()

In [6]:
#@title STEP5: Change file
%%writefile /content/CVPR2022-DaGAN/demo.py
import matplotlib
matplotlib.use('Agg')
import os, sys
import yaml
from argparse import ArgumentParser
from tqdm import tqdm
import modules.generator as GEN
import imageio
import numpy as np
from skimage.transform import resize
from skimage import img_as_ubyte
import torch
from sync_batchnorm import DataParallelWithCallback
import depth
from modules.keypoint_detector import KPDetector
from animate import normalize_kp
from scipy.spatial import ConvexHull
from collections import OrderedDict
import pdb
import cv2
if sys.version_info[0] < 3:
    raise Exception("You must use Python 3 or higher. Recommended version is Python 3.7")

def load_checkpoints(config_path, checkpoint_path, cpu=False):

    with open(config_path) as f:
        config = yaml.safe_load(f)
    if opt.kp_num != -1:
        config['model_params']['common_params']['num_kp'] = opt.kp_num
    generator = getattr(GEN, opt.generator)(**config['model_params']['generator_params'],**config['model_params']['common_params'])
    if not cpu:
        generator.cuda()
    config['model_params']['common_params']['num_channels'] = 4
    kp_detector = KPDetector(**config['model_params']['kp_detector_params'],
                             **config['model_params']['common_params'])
    if not cpu:
        kp_detector.cuda()
    if cpu:
        checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    else:
        checkpoint = torch.load(checkpoint_path,map_location="cuda:0")
        
    ckp_generator = OrderedDict((k.replace('module.',''),v) for k,v in checkpoint['generator'].items())
    generator.load_state_dict(ckp_generator)
    ckp_kp_detector = OrderedDict((k.replace('module.',''),v) for k,v in checkpoint['kp_detector'].items())
    kp_detector.load_state_dict(ckp_kp_detector)
    
    if not cpu:
        generator = DataParallelWithCallback(generator)
        kp_detector = DataParallelWithCallback(kp_detector)

    generator.eval()
    kp_detector.eval()
    
    return generator, kp_detector


def make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True, cpu=False):
    sources = []
    drivings = []
    with torch.no_grad():
        predictions = []
        depth_gray = []
        source = torch.tensor(source_image[np.newaxis].astype(np.float32)).permute(0, 3, 1, 2)
        driving = torch.tensor(np.array(driving_video)[np.newaxis].astype(np.float32)).permute(0, 4, 1, 2, 3)
        if not cpu:
            source = source.cuda()
            driving = driving.cuda()
        outputs = depth_decoder(depth_encoder(source))
        depth_source = outputs[("disp", 0)]

        outputs = depth_decoder(depth_encoder(driving[:, :, 0]))
        depth_driving = outputs[("disp", 0)]
        source_kp = torch.cat((source,depth_source),1)
        driving_kp = torch.cat((driving[:, :, 0],depth_driving),1)
       
        kp_source = kp_detector(source_kp)
        kp_driving_initial = kp_detector(driving_kp) 

        # kp_source = kp_detector(source)
        # kp_driving_initial = kp_detector(driving[:, :, 0])

        for frame_idx in tqdm(range(driving.shape[2])):
            driving_frame = driving[:, :, frame_idx]

            if not cpu:
                driving_frame = driving_frame.cuda()
            outputs = depth_decoder(depth_encoder(driving_frame))
            depth_map = outputs[("disp", 0)]

            gray_driving = np.transpose(depth_map.data.cpu().numpy(), [0, 2, 3, 1])[0]
            gray_driving = 1-gray_driving/np.max(gray_driving)

            frame = torch.cat((driving_frame,depth_map),1)
            kp_driving = kp_detector(frame)

            kp_norm = normalize_kp(kp_source=kp_source, kp_driving=kp_driving,
                                   kp_driving_initial=kp_driving_initial, use_relative_movement=relative,
                                   use_relative_jacobian=relative, adapt_movement_scale=adapt_movement_scale)
            out = generator(source, kp_source=kp_source, kp_driving=kp_norm,source_depth = depth_source, driving_depth = depth_map)

            drivings.append(np.transpose(driving_frame.data.cpu().numpy(), [0, 2, 3, 1])[0])
            sources.append(np.transpose(source.data.cpu().numpy(), [0, 2, 3, 1])[0])
            predictions.append(np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0])
            depth_gray.append(gray_driving)
    return sources, drivings, predictions,depth_gray


def find_best_frame(source, driving, cpu=False):
    import face_alignment

    def normalize_kp(kp):
        kp = kp - kp.mean(axis=0, keepdims=True)
        area = ConvexHull(kp[:, :2]).volume
        area = np.sqrt(area)
        kp[:, :2] = kp[:, :2] / area
        return kp

    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=True,
                                      device='cpu' if cpu else 'cuda')
    kp_source = fa.get_landmarks(255 * source)[0]
    kp_source = normalize_kp(kp_source)
    norm  = float('inf')
    frame_num = 0
    for i, image in tqdm(enumerate(driving)):
        kp_driving = fa.get_landmarks(255 * image)[0]
        kp_driving = normalize_kp(kp_driving)
        new_norm = (np.abs(kp_source - kp_driving) ** 2).sum()
        if new_norm < norm:
            norm = new_norm
            frame_num = i
    return frame_num

if __name__ == "__main__":
    parser = ArgumentParser()
    parser.add_argument("--config", required=True, help="path to config")
    parser.add_argument("--checkpoint", default='vox-cpk.pth.tar', help="path to checkpoint to restore")

    parser.add_argument("--source_image", default='sup-mat/source.png', help="path to source image")
    parser.add_argument("--driving_video", default='sup-mat/source.png', help="path to driving video")
    parser.add_argument("--result_video", default='result.mp4', help="path to output")
    
    parser.add_argument("--relative", dest="relative", action="store_true", help="use relative or absolute keypoint coordinates")
    parser.add_argument("--adapt_scale", dest="adapt_scale", action="store_true", help="adapt movement scale based on convex hull of keypoints")
    parser.add_argument("--generator", type=str, required=True)
    parser.add_argument("--kp_num", type=int, required=True)


    parser.add_argument("--find_best_frame", dest="find_best_frame", action="store_true", 
                        help="Generate from the frame that is the most alligned with source. (Only for faces, requires face_aligment lib)")

    parser.add_argument("--best_frame", dest="best_frame", type=int, default=None,  
                        help="Set frame to start from.")
 
    parser.add_argument("--cpu", dest="cpu", action="store_true", help="cpu mode.")
 

    parser.set_defaults(relative=False)
    parser.set_defaults(adapt_scale=False)

    opt = parser.parse_args()

    depth_encoder = depth.ResnetEncoder(18, False)
    depth_decoder = depth.DepthDecoder(num_ch_enc=depth_encoder.num_ch_enc, scales=range(4))
    loaded_dict_enc = torch.load('depth/models/weights_19/encoder.pth')
    loaded_dict_dec = torch.load('depth/models/weights_19/depth.pth')
    filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in depth_encoder.state_dict()}
    depth_encoder.load_state_dict(filtered_dict_enc)
    depth_decoder.load_state_dict(loaded_dict_dec)
    depth_encoder.eval()
    depth_decoder.eval()
    if not opt.cpu:
        depth_encoder.cuda()
        depth_decoder.cuda()

    source_image = imageio.imread(opt.source_image)
    reader = imageio.get_reader(opt.driving_video)
    fps = reader.get_meta_data()['fps']
    driving_video = []
    try:
        for im in reader:
            driving_video.append(im)
    except RuntimeError:
        pass
    reader.close()

    source_image = resize(source_image, (256, 256))[..., :3]
    driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
    generator, kp_detector = load_checkpoints(config_path=opt.config, checkpoint_path=opt.checkpoint, cpu=opt.cpu)

    if opt.find_best_frame or opt.best_frame is not None:
        i = opt.best_frame if opt.best_frame is not None else find_best_frame(source_image, driving_video, cpu=opt.cpu)
        print ("Best frame: " + str(i))
        driving_forward = driving_video[i:]
        driving_backward = driving_video[:(i+1)][::-1]
        sources_forward, drivings_forward, predictions_forward,depth_forward = make_animation(source_image, driving_forward, generator, kp_detector, relative=opt.relative, adapt_movement_scale=opt.adapt_scale, cpu=opt.cpu)
        sources_backward, drivings_backward, predictions_backward,depth_backward = make_animation(source_image, driving_backward, generator, kp_detector, relative=opt.relative, adapt_movement_scale=opt.adapt_scale, cpu=opt.cpu)
        predictions = predictions_backward[::-1] + predictions_forward[1:]
        sources = sources_backward[::-1] + sources_forward[1:]
        drivings = drivings_backward[::-1] + drivings_forward[1:]
        depth_gray = depth_backward[::-1] + depth_forward[1:]

    else:
        # predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=opt.relative, adapt_movement_scale=opt.adapt_scale, cpu=opt.cpu)
        sources, drivings, predictions,depth_gray = make_animation(source_image, driving_video, generator, kp_detector, relative=opt.relative, adapt_movement_scale=opt.adapt_scale, cpu=opt.cpu)
    imageio.mimsave('demo.mp4', [img_as_ubyte(p) for p in predictions], fps=fps)
    imageio.mimsave(opt.result_video, [np.concatenate((img_as_ubyte(s),img_as_ubyte(d),img_as_ubyte(p)),1) for (s,d,p) in zip(sources, drivings, predictions)], fps=fps)
    imageio.mimsave("gray.mp4", depth_gray, fps=fps)
    # merge the gray video
    animation = np.array(imageio.mimread(opt.result_video,memtest=False))
    gray = np.array(imageio.mimread("gray.mp4",memtest=False))

    src_dst = animation[:,:,:512,:]
    animate = animation[:,:,512:,:]
    merge = np.concatenate((src_dst,gray,animate),2)
    imageio.mimsave(opt.result_video, merge, fps=fps)
    #Transfer to gif
    # from moviepy.editor import *
    # clip = (VideoFileClip(opt.result_video))
    # clip.write_gif("{}.gif".format(opt.result_video))

Overwriting /content/CVPR2022-DaGAN/demo.py


In [ ]:
#@title STEP6: Animate it
kp_num = 15#@param {type:"integer"}
generator = "DepthAwareGenerator" #@param {type:"string"}
import os
%cd /content/CVPR2022-DaGAN
import os.path
from os import path
import yaml

!mkdir '/content/CVPR2022-DaGAN/depth/models/weights_19'
!cp '/content/CVPR2022-DaGAN/depth/CVPR22_DaGAN/depth_face_model/encoder.pth' '/content/CVPR2022-DaGAN/depth/models/weights_19'
!cp '/content/CVPR2022-DaGAN/depth/CVPR22_DaGAN/depth_face_model/depth.pth' '/content/CVPR2022-DaGAN/depth/models/weights_19'

!cd /content/CVPR2022-DaGAN && python demo.py --config "/content/CVPR2022-DaGAN/config/vox-adv-256.yaml" --driving_video "/content/fake.mp4" --source_image "/content/animate.png" --checkpoint "/content/CVPR2022-DaGAN/depth/CVPR22_DaGAN/DaGAN_vox_adv_256.pth.tar" --relative --adapt_scale --kp_num $kp_num --generator $generator

#Preview trimmed video
clear_output()
print("Trimmed Video")
showVideo('/content/CVPR2022-DaGAN/demo.mp4')